In [1]:
import tensorflow as tf
import numpy as np
import keras
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import keras.datasets as mnist
import os



2024-10-17 06:17:40.668865: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 06:17:40.668943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 06:17:40.670603: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 06:17:40.679570: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 06:17:42.198518: W tensorflow/compiler/tf2

In [15]:
#Image Data Details
IMAGE_SIZE = 28
IMAGE_PATCH_SIZE = 14
NUM_PATCHES = int((IMAGE_SIZE / IMAGE_PATCH_SIZE) ** 2)
NUM_CLASSES = 10
NUM_LAYERS = 1
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, NUM_LAYERS)

#Transformer settings
PROJECTION_HIDDEN_DIMENSIONS = int(98) #98 hidden dimensions when linear transforming for first time
NUM_SELFATTENTION_HEADS = 4 #each multihead has 4 self attention layers
TRANSFORMER_UNITS = [
    PROJECTION_HIDDEN_DIMENSIONS * 2,
    PROJECTION_HIDDEN_DIMENSIONS,
]  # Size of the transformer layers used for skip connecionts
TRANSFORMER_LAYERS = 16 #actual times transformer layers repeat
MLP_FINALLAYERS = [2028, 1024]

In [3]:
#load data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
#convert to float
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
#expand for said 1 layer dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print(x_train.shape)
print(y_train.shape)
print(y_train)

y_train = tf.convert_to_tensor(np.array(y_train), dtype="int32")
y_train = tf.convert_to_tensor(np.array(y_train), dtype="int32")
#print(y_train)
print(y_train.shape)
print(y_train)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28, 1)
(60000,)
[5 0 4 ... 5 6 8]


2024-10-17 06:17:45.393736: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


(60000,)
tf.Tensor([5 0 4 ... 5 6 8], shape=(60000,), dtype=int32)


2024-10-17 06:17:45.591944: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-17 06:17:45.592315: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-17 06:17:45.597560: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [27]:
#Hyper Params
learning_rate = 0.001
weight_decay = 0.000001
batch_size = 2000
num_epochs = 30

In [23]:
#Data Augmentation Layer
dataAugmentation = keras.Sequential([
    keras.layers.Normalization(),
    keras.layers.RandomRotation(.1),
    keras.layers.RandomZoom(width_factor=.1, height_factor=.1)
],
name="dataAug")
dataAugmentation.layers[0].adapt(x_train)
#plt.imshow(dataAugmentation(x_train[0]))
#print(x_train.shape)
#x = dataAugmentation(x_train[:2000])
#x = tf.reshape(x, shape=(2000, 28, 28, 1))
#print(x.shape)
#y = CreatePatches(IMAGE_PATCH_SIZE)(x)
#PositionalEmbeddingsAndEncoder(NUM_PATCHES, PROJECTION_HIDDEN_DIMENSIONS)(y)

In [17]:
#positional embeddings and creating patch sizes. Also final layer
def endPortionMLP(inputted, hiddenUnitsArray, dropoutRate):
    for units in hiddenUnitsArray:
        x = keras.layers.Dense(units, activation=tf.nn.gelu)(inputted)
        x = keras.layers.Dropout(dropoutRate)(x)
    return x

class CreatePatches(keras.layers.Layer):
    def __init__(self, patchSize):
        super().__init__()
        self.patchSize = patchSize
    
    def call(self, inputImages):
        batchSize = tf.shape(inputImages)[0] #gets the first column of input
        patches = tf.image.extract_patches(
            images = inputImages,
            sizes=[1, self.patchSize, self.patchSize, 1],
            strides=[1, self.patchSize, self.patchSize, 1],
            rates=[1, 1, 1, 1],
            padding="VALID" #this means no padding
        )
        #patchDimensions = patches.shape()[-1] #gets the shape minus number of patches in batch
        patches = tf.reshape(patches, [batchSize, -1, self.patchSize * self.patchSize* 1]) #shape now [batches, patch#, patchdims]
        return patches
    
class PositionalEmbeddingsAndEncoder(keras.layers.Layer):
    def __init__(self, numberPatches, projectionDimension):
        super().__init__()
        self.numPatches = numberPatches
        self.projection = keras.layers.Dense(units=projectionDimension)
        self.positionalEmbedding = keras.layers.Embedding(
            input_dim=numberPatches, output_dim=projectionDimension
        )
    
    def call(self, inputtedPatches):
        positions = tf.expand_dims(tf.range(start=0, limit=self.numPatches, delta=1), axis=0)
        #positions = tf.range(start=0, limit=self.numPatches, delta=1) #increment stuff
        embeddings = self.positionalEmbedding(positions) + self.projection(inputtedPatches)
        return embeddings
        #return self.positionalEmbedding(positions)
        #return self.projection(inputtedPatches)



In [25]:
#create the vision transformer
def VisionTransformer():
    #input layer
    inputs = keras.layers.Input(shape=x_train[0].shape)

    #get the augmented data
    augmentedData = dataAugmentation(inputs)
    
    #reshape bc tensorflow truncates for some reason
    augmentedData = tf.reshape(augmentedData, shape=(batch_size, IMAGE_SIZE, IMAGE_SIZE, 1))
    
    #get the individual patches
    patches = CreatePatches(IMAGE_PATCH_SIZE)(augmentedData) #init data then call

    encoded_patches = PositionalEmbeddingsAndEncoder(NUM_PATCHES, PROJECTION_HIDDEN_DIMENSIONS)(patches)

    #multihead attention blocks of encoder
    for i in range(TRANSFORMER_LAYERS):
        x1 = keras.layers.Normalization()(encoded_patches)

        attentionOutput = keras.layers.MultiHeadAttention(num_heads=NUM_SELFATTENTION_HEADS, key_dim=PROJECTION_HIDDEN_DIMENSIONS, dropout=0.12)(x1, x1)
        
        x2 = keras.layers.Add()([attentionOutput, encoded_patches]) #skip connection

        x3 = keras.layers.LayerNormalization(epsilon=1e-6)(x2)

        x3 = endPortionMLP(x3, hiddenUnitsArray=TRANSFORMER_UNITS, dropoutRate=0.12)

        encoded_patches = keras.layers.Add()([x3, x2]) #skip connection

    #[batchsize, info] tensor
    endingPortion = keras.layers.Normalization()(encoded_patches)
    endingPortion = keras.layers.Flatten()(endingPortion)
    endingPortion = keras.layers.Dropout(0.5)(endingPortion)

    #final mlp
    endingPortion = endPortionMLP(endingPortion, hiddenUnitsArray=TRANSFORMER_UNITS, dropoutRate=0.5)

    logits = keras.layers.Dense(NUM_CLASSES)(endingPortion)
    logits = keras.layers.Softmax()(logits)

    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [32]:
def runModel(model):
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate, weight_decay=weight_decay)
    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics=[keras.metrics.SparseCategoricalAccuracy(name='accuracy'), keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy")])
    
    checkpoint_path = "training_1/cp.ckpt.weights.h5"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    network_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 #monitor="accuracy",
                                                 #save_best_only=True
                                                )
    history = model.fit(x=x_train,y=y_train,batch_size=batch_size,epochs=num_epochs,validation_split=0.1,callbacks=[network_callback])

    model.load_weights(checkpoint_path)
    accuracy = model.evaluate(x_test, y_test)
    print("testing set accuracy")
    print(accuracy)
    return history

In [33]:
vit_classifier = VisionTransformer()
history = runModel(vit_classifier)


def plot_history(item):
    plt.plot(history.history[item], label=item)
    plt.plot(history.history["val_" + item], label="val_" + item)
    plt.xlabel("Epochs")
    plt.ylabel(item)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
    plt.legend()
    plt.grid()
    plt.show()


plot_history("loss")
plot_history("accuracy")

Epoch 1/30
27/27 [==============================] - 31s 286ms/step - loss: 5.0841 - accuracy: 0.3160 - top-5-accuracy: 0.7545 - val_loss: 0.9517 - val_accuracy: 0.7003 - val_top-5-accuracy: 0.9300
Epoch 2/30
27/27 [==============================] - 6s 232ms/step - loss: 1.6269 - accuracy: 0.5034 - top-5-accuracy: 0.8733 - val_loss: 0.7346 - val_accuracy: 0.7693 - val_top-5-accuracy: 0.9543
Epoch 3/30
27/27 [==============================] - 6s 233ms/step - loss: 1.2844 - accuracy: 0.6018 - top-5-accuracy: 0.9111 - val_loss: 0.5352 - val_accuracy: 0.8173 - val_top-5-accuracy: 0.9828
Epoch 4/30
27/27 [==============================] - 6s 235ms/step - loss: 0.8632 - accuracy: 0.7404 - top-5-accuracy: 0.9556 - val_loss: 0.2333 - val_accuracy: 0.9325 - val_top-5-accuracy: 0.9923
Epoch 5/30
27/27 [==============================] - 6s 234ms/step - loss: 0.6361 - accuracy: 0.8145 - top-5-accuracy: 0.9721 - val_loss: 0.1877 - val_accuracy: 0.9442 - val_top-5-accuracy: 0.9962
Epoch 6/30
27/27 [=

InvalidArgumentError: Graph execution error:

Detected at node model_7/positional_embeddings_and_encoder_7/dense_228/Tensordot/Reshape defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/ipykernel_launcher.py", line 17, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 542, in dispatch_queue